# Avoiding Road Accidents

## Dataset: 

http://vllab1.ucmerced.edu/~hhsu22/rear_signal/rear_signal#

### Dataset statistics

Total sequences: 649
Total frames: 63637

*Number of sequences in each class*:

OOO: 188 BOO: 211 OLO: 78 BLO: 63

OOR:  58 BOR:  33 OLR:  9 BLR:  9

*Number of frames in each class:*:


OOO: 21867 BOO: 17874 OLO: 6271 BLO: 6380

OOR:  4728 BOR:  3527 OLR: 1600 BLR: 1390

In [68]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current device: {torch.cuda.get_device_name(0)}")

CUDA available: True
Current device: NVIDIA GeForce RTX 3050 Laptop GPU


In [64]:
!pip install opencv-python
!pip install ultralytics

  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/899.1 kB ? eta -:--:--
   --------------------------------------- 899.1/899.1 kB 13.6 MB/s eta 0:00:00
Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [ ]:
import logging

# Setup logger
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import os
from pathlib import Path
from glob import glob
import shutil
import cv2

In [ ]:
os.makedirs("data/yolo_dataset/images/train", exist_ok=True)
os.makedirs("data/yolo_dataset/images/val", exist_ok=True)
os.makedirs("data/yolo_dataset/labels/train", exist_ok=True)
os.makedirs("data/yolo_dataset/labels/val", exist_ok=True)

In [ ]:
# Define class mapping
class_mapping = {
    'OOO': 0,  # Normal
    'BOO': 1,  # Braking
    'OLO': 2,  # Left signal
    'BLO': 3,  # Brake + Left signal
    'OOR': 4,  # Right signal
    'BOR': 5,  # Brake + Right signal
    'OLR': 6,  # Hazard lights
    'BLR': 7   # Brake + Hazard lights
}


In [ ]:
def process_images(footage_path, split='train'):
    # logger.info("Processing footage path: {footage_path}")

    # Get all frame paths - add light_mask to the path
    footage_path = Path(footage_path)  # Convert to Path if it isn't already
    frame_paths = list(footage_path.rglob('light_mask/frame*.png'))
    # logger.info(f"Found {len(frame_paths)} frames")

    for frame_path in frame_paths:
        # try:
        # Read image
        img = cv2.imread(str(frame_path))  # cv2.imread needs string path
        if img is None:
            logger.error(f"Failed to read image: {frame_path}")
            continue

        # Extract class from path
        frame_path_str = str(frame_path)  # Convert to string for pattern matching
        for class_name in class_mapping.keys():
            if f"_{class_name}_" in frame_path_str:
                break
        else:
            logger.error(f"Could not find class name in path: {frame_path}")
            continue

        # logger.info(f"Found class: {class_name}")

        # Generate filename
        filename = frame_path.name

        # Create destination paths using Path
        dest_path = Path(f"data/yolo_dataset/images/{split}/{filename}")
        label_path = Path(f"data/yolo_dataset/labels/{split}/{filename}").with_suffix('.txt')

        # logger.info(f"Copying to: {dest_path}")

        # Copy image
        shutil.copy2(str(frame_path), str(dest_path))

        # Create label file
        with open(label_path, 'w') as f:
            f.write(f'{class_mapping[class_name]} 0.5 0.8 0.3 0.2\n')

        # logger.info(f"Successfully processed: {filename}")

        # except Exception as e:
        #     logger.error(f"Error processing {frame_path}: {str(e)}")

In [ ]:
base_path = Path("data/rear_signal_dataset")

with open('data/rear_signal_dataset/Easy.txt', 'r') as f:
    easy_sequences = f.read().splitlines()
logger.info(f"Found {len(easy_sequences)} easy sequences")

# Split into train and validation
train_sequences = easy_sequences[:int(len(easy_sequences)*0.8)]
val_sequences = easy_sequences[int(len(easy_sequences)*0.8):]

logger.debug(f"Processing {len(train_sequences)} train sequences...")
for seq in train_sequences:
    # Reconstruct the correct path
    # Extract the base parts of the sequence name
    base_parts = seq.split('_')[:4]  # Get the first 4 parts
    base_name = '_'.join(base_parts)
    class_name = seq.split('_')[-2]  # Get class name (BOO, OOO, etc)
    sequence_num = seq.split('_')[-1]  # Get sequence number

    # Construct the full path
    seq_path = base_path.joinpath(base_name, f"{base_name}_{class_name}", f"{base_name}_{class_name}_{sequence_num}")
    # logger.debug(f"Looking for sequence at: {seq_path}")

    if os.path.exists(seq_path):
        process_images(seq_path, 'train')
    else:
        if base_parts[0].startswith('test'):
            complete_split = seq.split('_')
            if complete_split[2].startswith('idx'):
                base_parts = seq.split('_')[:3]
            else:
                base_parts = seq.split('_')[:2]
            base_name = '_'.join(base_parts)
            class_name = seq.split('_')[-2]
            sequence_num = seq.split('_')[-1]

            # construct full path for test
            seq_path = base_path.joinpath(base_name, f"{base_name}_{class_name}", f"{base_name}_{class_name}_{sequence_num}")
            # logger.debug(f"Looking for sequence at: {seq_path}")

            if os.path.exists(seq_path):
                process_images(seq_path, 'train')
            else:
                logger.error(f"Sequence path does not exist: {seq_path}")


logger.debug(f"Processing {len(val_sequences)} validation sequences...")
for seq in val_sequences:
    # logger.debug(f"Processing validation sequence: {seq}")
    base_parts = seq.split('_')[:4]
    base_name = '_'.join(base_parts)
    class_name = seq.split('_')[-2]
    sequence_num = seq.split('_')[-1]

    seq_path = base_path.joinpath(base_name, f"{base_name}_{class_name}", f"{base_name}_{class_name}_{sequence_num}")

    if os.path.exists(seq_path):
        process_images(seq_path, 'val')
    else:
        if base_parts[0].startswith('test'):
            complete_split = seq.split('_')
            if complete_split[2].startswith('idx'):
                base_parts = seq.split('_')[:3]
            else:
                base_parts = seq.split('_')[:2]
            base_name = '_'.join(base_parts)
            class_name = seq.split('_')[-2]
            sequence_num = seq.split('_')[-1]

            # Construct full path for test
            seq_path = base_path.joinpath(base_name, f"{base_name}_{class_name}", f"{base_name}_{class_name}_{sequence_num}")
            # logger.debug(f"Looking for sequence at: {seq_path}")

            if os.path.exists(seq_path):
                process_images(seq_path, 'val')
            else:
                logger.error(f"Sequence path does not exist: {seq_path}")


# Print final statistics
train_images = len(glob('data/yolo_dataset/images/train/*.png'))
val_images = len(glob('data/yolo_dataset/images/val/*.png'))

logger.info(f"Final Statistics:")
logger.info(f"Training images: {train_images}")
logger.info(f"Validation images: {val_images}")


In [66]:
# Create yaml file first
yaml_content = f"""
path: {os.path.abspath('data/yolo_dataset')}  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')

# Classes
nc: {len(class_mapping)}  # number of classes
names: {list(class_mapping.keys())}  # class names
"""

with open('dataset.yaml', 'w') as f:
    f.write(yaml_content)

In [71]:
!pip uninstall -y ultralytics
!pip install torch torchvision ultralytics==8.0.196


Found existing installation: ultralytics 8.0.196
Uninstalling ultralytics-8.0.196:
  Successfully uninstalled ultralytics-8.0.196
  Using cached ultralytics-8.0.196-py3-none-any.whl.metadata (31 kB)
Using cached ultralytics-8.0.196-py3-none-any.whl (631 kB)


In [72]:
!yolo task=detect mode=train model=yolov8n.pt data=dataset.yaml epochs=5 imgsz=640 batch=16 device=cuda



^C
